# Student Grade Prediction Model

## Authors:
- Samuel Nicklaus
- Jacob Kindle
- Lane Swartzendruber
- Alejandro Mirafuentes

## Introduction

In this project, we tackle the [classification/regression] task of [briefly describe the task]. Our dataset is sourced from [mention data source here, if applicable]. We aim to [briefly explain the objective of the task].

In [11]:
# imports
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import seaborn as sns
from sklearn import (datasets, neighbors,
                     naive_bayes,
                     model_selection as skms,
                     linear_model, dummy,
                     metrics,
                     pipeline,
                     preprocessing as skpre)
import ipywidgets as widgets
from ipywidgets import interact

# Data Description

Talk about data here

In [12]:
df = pd.read_csv('student.csv')
df.drop(['Unnamed: 0', 'Id'], axis=1, inplace=True)
df.head()


,Student_Age,Sex,High_School_Type,Scholarship,Additional_Work,Sports_activity,Transportation,Weekly_Study_Hours,Attendance,Reading,Notes,Listening_in_Class,Project_work,Grade
0,21,Male,Other,50%,Yes,No,Private,0,Always,Yes,Yes,No,No,AA
1,20,Male,Other,50%,Yes,No,Private,0,Always,Yes,No,Yes,Yes,AA
2,21,Male,State,50%,No,No,Private,2,Never,No,No,No,Yes,AA
3,18,Female,Private,50%,Yes,No,Bus,2,Always,No,Yes,No,No,AA
4,22,Male,Private,50%,No,No,Bus,12,Always,Yes,No,Yes,Yes,AA
